In [1]:
# What allows us to make changes to the z2jh_cost_simulator package
# and workaround a caching of the module, so our changes can be seen.
%load_ext autoreload
%autoreload 2

In [2]:
from z2jh_cost_simulator import simulator

In [ ]:
# Now we can work with the simulator object.
# It is time to transition the code from the notebook!